In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df = pd.read_csv('zkp_repos.csv', sep=';')
commit_df = pd.read_csv('tool_commits.csv')
contributors_df = pd.read_csv('contributor_data.csv')

#### Preprocess Data

In [ ]:
df = df.rename({"Tool Resources (Twitter, Discord, Website etc.)": "Tool Resources"}, axis='columns')
df['UniqueID'] = df['URL'].apply(lambda x: '/'.join(x.split('/')[-2:][::-1]).lower())

commit_df['UniqueID'] = commit_df['URL'].apply(lambda x: '/'.join(x.split('/')[-2:][::-1]).lower())

df_applications = df[df['Type'] == 'Application']

df_applications['Tool'] = df_applications['Tool'].str.split(', ')
df_applications['Tool'] = df_applications['Tool'].apply(lambda x: [item for item in x if item != ''])


#### Relationship Between Tool Use and External Resources Available for Tool

In [ ]:
df_applications = df[df['Type'] == 'Application']

df_applications['Tool'] = df_applications['Tool'].str.split(', ')
df_applications['Tool'] = df_applications['Tool'].apply(lambda x: [item for item in x if item != ''])

expanded_df = df_applications.explode('Tool')

tool_counts = expanded_df['Tool'].value_counts().reset_index()
tool_counts.columns = ['Tool', 'Frequency']

tool_resources = df[df['Type'] == 'Tool']
tool_resources['Name'] = tool_resources['Name'].str.lower()
tool_resources = tool_resources.groupby('Name')['Tool Resources'].max().reset_index()
tool_resources.columns = ['Tool', 'Tool Resources']
tool_data = pd.merge(tool_counts, tool_resources, on='Tool', how='left')


fig = px.bar(tool_data, x='Tool', y='Frequency',
             title='Relationship Between Tool Frequency and External Resources',
             labels={'Frequency': 'Tool Frequency', 'Tool Resources': 'Tool Resources Available'},
             template="plotly_dark",
             color='Tool Resources',
             text='Frequency',
             category_orders={"Tool": tool_data["Tool"].tolist()}
             )

fig.update_layout(legend_title_text='Tool Resources')
fig.show()


#### All Commits Over Time for Various Repositories

In [ ]:
all_commits_df = commit_df.sort_values(by='CommitterDate')
all_commits_df['UniqueID'] =( all_commits_df['Name'] + '/' + all_commits_df['Owner']).str.lower()
all_commits_df['CommitterDate'] = pd.to_datetime(all_commits_df['CommitterDate'])
fig = px.scatter(all_commits_df, x='CommitterDate', y='UniqueID', color='UniqueID',
                 title='All Commits Over Time for Various Repositories',
                 labels={'Date': 'Commit Date', 'Name': 'Repository'},
                 template="plotly_dark")

fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Repository')
fig.update_layout(legend_title_text='Repository', height=1000)
fig.show()


#### Committer Count per Tool Repository

In [ ]:
committers_df = commit_df
committers_df['UniqueID'] = commit_df['Name'] + '/' + commit_df['Owner']
committers_df['Committer'] = committers_df['Committer'].str.lower()
author_counts = committers_df.groupby('UniqueID')['Committer'].nunique().reset_index()
author_counts.rename(columns={'Committer': 'CommitterCount'}, inplace=True)
author_counts.sort_values(by=['CommitterCount'], ascending=False, inplace=True)

fig = px.bar(author_counts, x='UniqueID', y='CommitterCount', 
             title='Committer Count per Repository',
             template="plotly_dark",
             text='CommitterCount',
             labels={'CommitterCount': 'No. of Committers', 'UniqueID': 'Repository'},)
fig.show()


#### Contributor Count per Tool Repository

In [ ]:
contributors_df['UniqueID'] = contributors_df['RepositoryName'] + '/' + contributors_df['Owner']
contributors_df = contributors_df[contributors_df['UniqueID'].isin(committers_df['UniqueID'])]
contributors_df['Contributor'] = contributors_df['Contributor'].str.lower()
contributor_counts = contributors_df.groupby('UniqueID')['Contributor'].nunique().reset_index()
contributor_counts.rename(columns={'Contributor': 'ContributorCount'}, inplace=True)
contributor_counts.sort_values(by=['ContributorCount'], ascending=False, inplace=True)


fig = px.bar(contributor_counts, x='UniqueID', y='ContributorCount', 
             title='Contributor Count per Repository',
             template="plotly_dark",
             text='ContributorCount',
             labels={'ContributorCount': 'No. of Contributors', 'UniqueID': 'Repository'}
             )
fig.show()


In [ ]:
author_counts

In [ ]:
df_applications = df[df['Type'] == 'Application']

df_applications['Tool'] = df_applications['Tool'].str.split(', ')
df_applications['Tool'] = df_applications['Tool'].apply(lambda x: [item for item in x if item != ''])


df_tools = df[df['Type'] == 'Tool']
# tool_resources['Name'] = tool_resources['Name'].str.lower()
df_tool_resources = df_tools.groupby(['Name', 'UniqueID'])['Tool Resources'].max().reset_index()
df_tool_resources.columns = ['Name', 'UniqueID', 'Tool Resources']
df_tool_resources

In [ ]:
all_commits_df.sample(1).T

In [ ]:
all_commits_df

In [ ]:
df_tools_vs_commits = tool_data[['Tool', 'Frequency']].merge(df[['Name', 'UniqueID']], left_on='Tool', right_on='Name', how='left').merge(all_commits_df, on='UniqueID', how='left')

In [ ]:
df_tools_vs_commits['Committer'].value_counts()

In [ ]:
px.scatter(
    df_tools_vs_commits.groupby(["UniqueID", "Tool", "Frequency"])
    .agg({"Committer": "nunique"})
    .reset_index()
    .rename(columns={"Committer": "# Of Commiters", "Frequency": "# Apps Used"}),
    x="# Of Commiters",
    y="# Apps Used",
    # color="Tool",
    size="# Of Commiters",
    template="plotly_dark",
    title="Relationship Between Number of Committers and Number of Apps Used",
    labels={
        "# Of Commiters": "Number of Committers",
        "# Apps Used": "Number of Apps Used",
    },
    hover_name="UniqueID",
    width=500,
    height=500,
    # ols
    trendline="ols",
)

In [ ]:
# ['Tool', 'Frequency', 'Name_x', 'UniqueID', 'Unnamed: 0', 'Name_y',
#        'Owner', 'CommitHash', 'Message', 'Author', 'AuthorEmail', 'Committer',
#        'CommitterEmail', 'AuthorDate', 'CommitterDate', 'AuthorTimeZone',
#        'CommitterTimeZone', 'Branches', 'Main', 'Merge', 'ModificationCount',
#        'AddedFiles', 'ModifiedFiles', 'DeletedFiles', 'RenamedFiles',
#        'Parents', 'Deletions', 'Insertions', 'Lines', 'Files'],
df_tools_vs_commits.columns

In [ ]:
df_tools_vs_commits['Branches'].unique()

In [ ]:
df

In [ ]:
all_commits_df.sample(3).T

In [ ]:
all_commits_df["CommitterDateMin"] = all_commits_df["CommitterDate"]
all_commits_df["CommitterDateMax"] = all_commits_df["CommitterDate"]
df_commit_features = (
    all_commits_df.groupby("UniqueID")
    .agg(
        {
            "CommitterDateMin": "min",
            "CommitterDateMax": "max",
            "CommitHash": "nunique",
            "Committer": "nunique",
            "Lines": "mean",
            "Files": "mean",
            "Deletions": "mean",
            "Insertions": "mean",
        }
    )
    .reset_index()
    .rename(
        columns={
            "CommitHash": "CommitCount",
            "Committer": "CommitterCount",
            "Lines": "AvgLines",
            "Files": "AvgFiles",
            "Deletions": "AvgDeletions",
            "Insertions": "AvgInsertions",
        }
    )
)
df_commit_features['UniqueID'].nunique()

In [ ]:
df['UniqueID'].nunique()    

In [ ]:
df_git_features = df[
    [
        "Name",
        "UniqueID",
        "Size",
        "Language",
        "Stars",
        "Forks",
        "Watchers",
        "Issues",
        "Type",
    ]
]
df_git_features["Name"] = df_git_features["Name"].str.lower()
df_tools_features = (
    df_git_features[df_git_features["Type"] == "Tool"]
    .merge(df_commit_features, on="UniqueID", how="outer")
    .merge(
        tool_counts.merge(
            df[["Name", "UniqueID"]], left_on="Tool", right_on="Name", how="left"
        )[["UniqueID", "Frequency"]].rename(columns={"Frequency": "AppCount"}),
        on="UniqueID",
        how="left",
    )
)
df_tools_features.sort_values(by="CommitterDateMin", ascending=False)
df_tools_features['LifetimeDays'] = (df_tools_features['CommitterDateMax'] - df_tools_features['CommitterDateMin']).apply(lambda x: x.days)
# df_tools_features = pd.concat([df_tools_features, pd.get_dummies(df_tools_features['Language'])], axis=1)
df_tools_features = df_tools_features.drop(['Name', 'CommitterDateMin', 'CommitterDateMax', 'Type'] , axis=1)
# one hot encode Language
df_tools_features.sample(2).T

In [ ]:
px.violin(df_tools_features.melt(id_vars=['UniqueID'], value_vars=['Stars', 'Forks', 'Watchers', 'Issues', 'CommitCount', 'CommitterCount', 'AvgLines', 'AvgFiles', 'AvgDeletions', 'AvgInsertions', 'AppCount', 'LifetimeDays']),
            x='variable',
            y='value',
            color='variable',
            template="plotly_dark",
            title='Distribution of Git Features',
            labels={'variable': 'Feature', 'value': 'Value'},
            width=1000,
            height=500
            )

In [ ]:

px.violin(df_tools_features, y="AvgLines", x="Language", box=True, points="all")

In [ ]:
low_cutoff = list(df_tools_features["AppCount"].quantile([0.33, 0.66]).values)[0]
med_cutoff = list(df_tools_features["AppCount"].quantile([0.33, 0.66]).values)[1]
low_cutoff, med_cutoff

In [ ]:
# create low, medium and high frmo app count based on quantiles


df_tools_features["UsageGroup"] = df_tools_features["AppCount"].apply(
    lambda x: "low" if x <= low_cutoff else "medium" if x <= med_cutoff else "high"
)
df_tools_features.sort_values(by="AppCount", ascending=False)

In [ ]:
# ? zksync/matter-labs	383037.0	Rust	3376.0	2305.0	119.0	55.0	12193	56	1339.918724	14.687608	500.743787	839.174936	4	1554	low

In [ ]:
# cluster tools
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

cols = [
    # "UniqueID",

    # Perception Features
    # "Size",
    "Stars",
    "Forks",
    "Watchers",
    "Issues",

    # Code Base Features
    "CommitCount",
    "CommitterCount",
    "AvgLines",
    "AvgFiles",
    "AvgDeletions",
    "AvgInsertions",
    "LifetimeDays",

    # "AppCount",
    
    # "C++",
    # "Go",
    # "Haskell",
    # "JavaScript",
    # "OCaml",
    # "Python",
    # "Rust",
    # "WebAssembly",
]

nr_components = 2

X = df_tools_features[cols]
scaler = StandardScaler()
scaler.fit(X)
df_tools_features_scaled = scaler.transform(X)
pca = PCA(n_components=nr_components)
pca.fit(df_tools_features_scaled)
df_tools_features_pca = pca.transform(df_tools_features_scaled)
df_tools_features_pca = pd.DataFrame(df_tools_features_pca)
# df_tools_features_pca.columns = ["PCA1", "PCA2"]
df_tools_features_pca["UniqueID"] = df_tools_features["UniqueID"]
df_tools_features_pca = df_tools_features_pca.merge(
    df_tools_features, on="UniqueID", how="left"
)
components = pca.fit_transform(X)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
px.scatter(
    df_tools_features_pca,
    x=0,
    y=1,
    color="UsageGroup",
    hover_name="UniqueID",
    template="plotly_dark",
    title="PCA of Tool Features",
    labels={"0": "PCA1", "1": "PCA2"},
    width=500,
    height=500,
)

In [ ]:
import numpy as np
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

loading_matrix = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=cols)
loading_matrix

In [ ]:

fig = px.scatter_matrix(
 components,
    labels=df_tools_features_pca['UniqueID'],
    dimensions=range(nr_components),
    color=df_tools_features_pca["Language"],
    height=1000,
    template="plotly_dark",
    width=1000,
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
px.scatter(df_tools_features,
           x='CommitCount',
           y='CommitterCount',
           color='Language',

           template="plotly_dark",
           height=500,
           width=500,
           )